In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, make_scorer
from sklearn.tree import DecisionTreeClassifier as skDecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.tree import plot_tree
from my_ml_package.decision_tree import DecisionTreeClassifier
from my_ml_package.data.synthetic import variance_sample_for_decision_tree

## Delibrarely Generating A High-Variance Model

In [6]:

X_train, y_train = variance_sample_for_decision_tree(sample_size = 40) # we need some variabiliy in the data for bootstrap to work
X_test, y_test = variance_sample_for_decision_tree(sample_size = 2000)

treeclf = DecisionTreeClassifier()
treeclf.fit(X_train, y_train)
y_train_pred = treeclf.predict(X_train)
y_pred = treeclf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(accuracy_score(y_train, y_train_pred)))
print('The accuracy on test data is {:.2f}'.format(accuracy_score(y_test, y_pred)))
# print('The test error rate is {:.2f}'.format(1 - accuracy_score(y_test, y_pred)))


# The accuracy on training data is 0.85
# The accuracy on test data is 0.69

# 0.37
# 0.16

The accuracy on training data is 1.00
The accuracy on test data is 0.65


## Deal with High Variance 

Training an Ensemble model: train different independent models on slightly different subsets of data
* How to make each model independent with others? 
* Hint: The way the data is fed into the models can be challenging

## Bagging
* Trained Multiple Models on Bootstrap datasets
    + Bootstrap: Resampling the same size of sample with replacement; reduce variance
    + Bootstrap AGGregation (BAGGing): agggregate the prediction over a collection of bootstrap samples
    + A Bootstrap sample $\mathbf{Z}^{* b}, b=1,2, \ldots, B$ -> a fitted model $\hat{f}^{* b}(x)$
        $$\hat{f}_{\mathrm{bag}}(x)=\frac{1}{B} \sum_{b=1}^B \hat{f}^{* b}(x)$$

Bagging Inference
* Voting for classification: $\hat{G}_{\text {bag }}(x)=\arg \max _k \hat{f}_{\mathrm{bag}}(x)$
    + "It is tempting to treat the voting proportions pk(x) as estimates of these probabilities. A simple two-class example shows that they fail in this regard." (Hastie, 2008) **How/why fails?**
     <!--  Suppose the true probability of class 1 at x is 0.75, and each of the bagged classifiers accurately predict a 1. Then p1(x) = 1, which is incorrect. -->
    + "An alternative bagging strategy is to average these instead, rather than the vote indicator vectors."
* Averaging for regression 
* Out-of-bag samples: about 1/3 original data is not in the bootstrap dataset which can be used for model evaluation

Goal: reduce the variance of unstable (high variance) learning methods. Assuming that the variables are simply i.d. (identically distributed, but not necessarily independent) with positive pairwise correlation ρ, the variance
of the average is
\begin{equation}
\rho \sigma^2+\frac{1-\rho}{B} \sigma^2
(\#eq:variance)
\end{equation}

<!-- Question 1: Which learning model/method is ideal for bagging?

Question 2: Will it reduce bias?

"since each tree generated in bagging is [identically distributed (i.d.)](https://stats.stackexchange.com/questions/89036/why-the-trees-generated-via-bagging-are-identically-distributed#:~:text=Bagging%20technique%20uses%20bootstraps%20\(random,population%20as%20the%20original%20sample.), the expectation of an average of B such trees is the same as the expectation of any one of them. This means the bias of bagged trees is the same as that of the individual trees, and the only hope of improvement is through variance reduction. This is in contrast to boosting, where the trees are grown in an adaptive way to remove bias, and hence are not i.d." (Hastie, 2008) -->


<!-- <center><img src="pics/bagging.png" width="500"></center>

<center><img src="pics/bagging_result.png" width="500"></center>

* the trees have high variance due to the correlation in the predictors
* Bagging succeeds in smoothing out this variance and hence reducing the test error
    + " averaging reduces variance and leaves bias unchanged" (Hastie, 2008) -->






<!-- ## [Random Forest](https://link.springer.com/article/10.1023/A:1010933404324)
* "the size of the correlation of pairs of bagged trees limits the benefits of averaging" according to Formula 1
* "a substantial modification of bagging that builds a large collection of de-correlated trees"
* Iteratively 1) make a bootstrapped dataset; 2) only use a random subset of variables at each splitting (`max_features`)
* can handle large data sets with higher dimensionality (thousands of input variables).
* can identify most significant variables -->




In [7]:
# bootstrap sample
def boostrap(X, y):
    """ Returns a boostrap sample of the data X, y.

    returns:
    X_boot: np.array, shape (n_samples, n_features)
    y_boot: np.array, shape (n_samples, )
    """
    X_boot = []
    y_boot = []
    train_size = X.shape[0]
    for _ in range(train_size):
        idx = np.random.randint(0, len(X))
        X_boot.append(X[idx])
        y_boot.append(y[idx])
    # to numpy array
    return np.array(X_boot), np.array(y_boot)

# y_true = 0 # 3-class
# Orig: 30  -> Orig tree     y_test = 1 
# Bootstrap 1: 30 -> tree 1  y_test = 0
# Bootstrap 2: 30 -> tree 2  y_test = 1
# Bootstrap 3: 30 -> tree 3  y_test = 2
# Bootstrap 4: 30 -> tree 4  y_test = 0
# Bootstrap 5: 30 -> tree 5  y_test = 0
# 1: 2
# 0: 3

n_estimators = 2000
max_depth = 2
criterion = "entropy"
ensemble_clf = []
for _ in range(n_estimators):
    # bootstrap sample
    X_boot, y_boot = boostrap(X_train, y_train)

    # fit an ensemble of classification trees
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(X_boot, y_boot)
    ensemble_clf.append(clf)
   
# predict the test data
y_preds = []
for clf in ensemble_clf:
    y_preds.append(clf.predict(X_test))
y_pred = (np.array(y_preds).mean(axis=0) > 0.5).astype(int)
y_train_preds = []
for clf in ensemble_clf:
    y_train_preds.append(clf.predict(X_train))
y_train_pred = (np.array(y_train_preds).mean(axis=0) > 0.5).astype(int)

print('The accuracy on training data is {:.2f}'.format(accuracy_score(y_train, y_train_pred)))
print('The accuracy on test data is {:.2f}'.format(accuracy_score(y_test, y_pred)))

# sklearn implementation
rf_clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_features=None, max_depth=max_depth)   
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(rf_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(rf_clf.score(X_test, y_test)))


# for estimator in rf_clf.estimators_:
#     plt.figure(figsize=(12,12))
#     tree.plot_tree(estimator, filled=True, rounded=True)
#     plt.show()


The accuracy on training data is 0.78
The accuracy on test data is 0.50
The accuracy on training data is 0.88
The accuracy on test data is 0.61


In [9]:
# sklearn implementation
n_estimators = 2000
rf_clf = RandomForestClassifier(n_estimators=n_estimators, criterion="entropy", max_features=None, max_depth=2)   
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(rf_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(rf_clf.score(X_test, y_test)))


# for estimator in rf_clf.estimators_:
#     plt.figure(figsize=(12,12))
#     tree.plot_tree(estimator, filled=True, rounded=True)
#     plt.show()


The accuracy on training data is 0.85
The accuracy on test data is 0.75


[Looking for good `n_estimators`](https://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html)

## Boosting


* GradientBoost
    + an fixeds-size estimator normally has 8 to 32 leaves
    + iteratively fit residuals by a split
    + use learning rate to avoid high bias
    + [Youtube Course](vhttps://www.youtube.com/watch?v=3CC4N4z3GJc)

In [ ]:
import numpy as np

class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []
        self.initial_probs = None

    def fit(self, X, y):
        # Initialize probabilities for binary classification
        self.initial_probs = np.full(y.shape, np.mean(y)) # y= np.array([1, 0, 0]) -> array([0.33333333, 0.33333333, 0.33333333])

        # Convert to log-odds for starting point
        initial_log_odds = np.log(self.initial_probs / (1 - self.initial_probs))
        current_logits = np.full(y.shape, initial_log_odds)
        
        for _ in range(self.n_estimators):
            # Compute residuals (negative gradients of log-loss)
            probabilities = 1 / (1 + np.exp(-current_logits))
            gradients = y - probabilities
            
            # Train a decision tree on the gradients
            tree = DecisionTreeClassifier(max_depth=self.max_depth)
            tree.fit(X, gradients)
            self.models.append(tree)
            
            # Update the predictions
            current_logits += self.learning_rate * tree.predict_proba(X)[:, 1]

    def predict_proba(self, X):
        # Start with the initial probabilities
        current_logits = np.full(X.shape[0], np.log(self.initial_probs / (1 - self.initial_probs)))
        
        # Add contributions from each trained tree
        for tree in self.models:
            current_logits += self.learning_rate * tree.predict_proba(X)[:, 1]
        
        # Convert back to probability using logistic function
        probabilities = 1 / (1 + np.exp(-current_logits))
        return np.vstack([1 - probabilities, probabilities]).T

    def predict(self, X):
        # Return class labels (0 or 1) based on predicted probabilities
        return np.argmax(self.predict_proba(X), axis=1)


<center><img src="pics/gradient-boost1.png" width="500"></center>

<center><img src="pics/gradient-boost2.png" width="500"></center>

In [131]:
# build a Gradient Boosting classifier with 200 trees
gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.5, max_depth=1, random_state=0)
gb_clf.fit(X_train, y_train)
y_pred = gb_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(gb_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(gb_clf.score(X_test, y_test)))
print('The test error rate is {:.2f}'.format(1 - gb_clf.score(X_test, y_test)))
print(classification_report(y_test, y_pred))



The accuracy on training data is 1.00
The accuracy on test data is 0.69
The test error rate is 0.31
              precision    recall  f1-score   support

         0.0       0.76      0.71      0.73      1197
         1.0       0.60      0.66      0.63       803

    accuracy                           0.69      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.69      0.69      0.69      2000



* AdaBoost
<!-- 
    + an fix-sized estimator uses only one feature, i.e., one stump (one root node with two leaf nodes)
    + weak learner
    + build the subsequent stumps using the residuals
    + the amount of say 
        $$\alpha_m=\log \left(\left(1-\operatorname{err}_m\right) / \operatorname{err}_m\right)$$

    + Update the weights
        $$w_i \cdot \exp \left[\alpha_m \cdot I\left(y_i \neq G_m\left(x_i\right)\right)\right], i=1,2, \ldots, N$$ -->
<!-- Emphasize the need to correctly classify the examples with wrong predictions in the previous steps -->

<!-- ```
# build an Adaboost classifier with 200 trees
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(ada_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(ada_clf.score(X_test, y_test)))
print('The test error rate is {:.2f}'.format(1 - ada_clf.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
``` -->
